# Building a CPI Bid Model (Part 1)
## Introduction
Beeswax has recently released a feature called Bid Models, which allows customer to create models that determine the exact bid by any combination of variables available on a programatic bid request. The idea is that, as a Beeswax user, you can leverage your existing logs (auctions, bids, wins, conversions, etc) to build a model that predicts the likelihood of some metric such as a click-through or app install.  That prediction can then be used to determine how much you should bid on each auction.  In general Bid Models are meant to be flexible, allowing you to use any modeling tools/workflow and express the model in a straight forward format.  A typical workflow might look like the following:

1. Create a model using Beeswax logs (wins, conversions, etc) and/or proprietary data.  Model features are bid request keys, and the prediction is either a bid or a bid multiplier.
2. Run a bunch of auction logs through the model to get a predicted bid price for each auction and create a set of prediction files from these predictions.
2. Create a Bid Model via Beeswax's API or UI, upload the predictions to Amazon S3, and tell Beeswax where they are.
3. Attach the Bid Model to a campaign or line item.
4. When a bid request reaches the bidding step for that line item, Beeswax does a lookup against the Bid Model. If there is a match, that value is used as the bid or multiplier.  If there is no match, the default bid is used.
5. As the campaign runs, use the performance data to retrain the model and upload new versions

In this tutorial, we will go through these steps end-to-end, including building a machine learning model with RTB logs and deploying it as a Bid Model on Beeswax. I'm going to leverage [Amazon Sagemaker](https://aws.amazon.com/sagemaker/) and the popular Pandas python library to build my model.  This will give me a Jupyter notebook environment managed by AWS with most of the networking and data access permissions I need.  That said, the techniques applied in this tutorial are pretty general and should apply to the modeling environment of your choice.  

Ultimately, we need to produce two assets in order to run our Bid Model on Beeswax:
* a "manifest" file that describes our model
* a set of "prediction" files that contain the actual data for our model.  The exact size and shape of these files will vary depending on your model but a simple example could look something like this:

| app_bundle | display_manager | placement_type   | banner_height | platform_os_version | value          |
|------------|-----------------|------------------|---------------|---------------------|----------------|
| 1005765746 | Fyber           | BANNER           | 320           | 4.1                 | [expected bid ]|
| 1008508212 | SOMA            | BANNER           | 320           | 11.0                | [expected bid ]|
| 1016562846 | AerservSDKiOS   | BANNER_AND_VIDEO | null          | 12.0                | [expected bid ]|
| ...        | ...             | ...              | ...           | ...                 | ...            |

For this exercise we are going to build a Cost-Per-Install (CPI) model, which will bid more on users who are more likely to download our mobile app. The purpose of this tutorial is not to go in-depth on CPI prediction so we are going to make some basic assumptions to simplify things:
* we will assume every install has the same value
* we will ignore auction dynamics like bid floors, market pricing, first vs second price etc
* our model will be general enough to use with any campaign, and not based on performance data from one particular campaign or set of creatives

To start, we are going to use the following simple equation to calculate bid price:

>conversion_value * likelihood_of_conversion = bid_price

In this scenario, our "conversion event" is an app install, so our model will predict the likelihood that a particular auction will lead to an app download. We just need to know how much each install is worth to us and then we can calculate our bid. For the purpose of this exercise, let's assume that we've done some research and determined that the lifetime value of a single app install is $5 (our conversion value).  In other words, our goal is to build a model that can achieve a CPI of <$5.

Let's get started.

## Data Gathering
In order to train a model, we need some data.  Since we ultimately want to use our model as a Beeswax Bid Model, we will use three Beeswax datasets to build the model:

* Win Logs: We'll use these to provide the features (aka the independent variables) for our model
* Conversion Logs: We'll join these to the Win Logs to provide the dependent variable (i.e. app installs)
* Auction Logs: Once we have a model ready to go, we'll run auction logs through the model to validate it and create the prediction files we will ultimately upload to Beeswax.  This will be done later in the tutorial.

For now, let's start with the Win and Conversion Logs.  Beeswax logs are usually stored in S3, so we will use the AWS client provided by boto to load them into a dataframe for further manipulation.

In [ ]:
import boto3
import numpy as np
import pandas as pd
import s3fs
import time

In [ ]:
BUCKET = 'beeswax-logs-us-east-1'
WIN_LOGS_PATH = 'bid-models-test-data/canary/win/'
CONVERSION_LOGS_PATH = 'bid-models-test-data/canary/conversion/'
NUMFILES = -80000

client = boto3.client('s3')
resource = boto3.resource('s3')
bucket = resource.Bucket(BUCKET)

fs = s3fs.S3FileSystem()

# we want to limit the size of the training dataset to make sure we can actually read
# all the data without blowing up the machine.  This will give us the most recent N
# files, and can be adjusted based on available memory and willingness to sit and wait
win_files = list(bucket.objects.filter(Prefix=WIN_LOGS_PATH))[NUMFILES:]
conversion_files = list(bucket.objects.filter(Prefix=CONVERSION_LOGS_PATH))[NUMFILES:]

We'll start by reading the win logs and, to keep the footprint small, we'll extract only the columns that are supported in Bid Models, plus `auction_id` and `bid_time_epoch_in_usecs` which we will use for transformations later:

In [ ]:
win_frames = []
for _file in win_files:
    if not _file.key.endswith('gz'):
        continue
    with fs.open('s3://{}/{}'.format(BUCKET, _file.key)) as f:
        df = pd.read_csv(f, compression='gzip', header=0, sep=',', quotechar='"')
        win_frames.append(df[['auction_id','ad_position','app_bundle','app_id','app_name','auction_type',
                 'platform_bandwidth', 'banner_height','banner_width','platform_browser',
                 'platform_browser_version','platform_carrier','geo_city','content_rating',
                 'content_coppa_flag','geo_country','platform_device_make','platform_device_model',
                 'platform_device_screen_size','platform_device_type','display_manager',
                 'display_manager_ver','domain','environment_type','inventory_interstitial',
                 'inventory_source','platform_js','content_language','geo_metro','platform_os',
                 'platform_os_version','placement','placement_type','publisher_id','geo_region',
                 'site_name','site_id','geo_zip', 'exchange_predicted_view_rate', 'rewarded', 
                 'video_boxing_allowed', 'video_companion_required','geo_lat', 'geo_lon', 'video_playback_method',
                 'video_player_size', 'video_start_delay', 'bid_time_epoch_in_usecs']])

win_df = pd.concat(win_frames, axis=0, ignore_index=True)
del win_frames  # make sure you delete these so they aren't sitting in memory

Now we'll do the same for Conversion logs. We really just need the `auction_id` so we can join this in to the Win logs.  We'll also filter down to the events that signify an app open as this will be the actual conversion event we care about

In [ ]:
conversion_frames = []
for _file in conversion_files:
    if not _file.key.endswith('gz'):
        continue
    with fs.open('s3://{}/{}'.format(BUCKET, _file.key)) as f:
        df = pd.read_csv(f, compression='gzip', header=0, sep=',', quotechar='"')
        conversion_frames.append(df[['auction_id']])

conversion_df = pd.concat(conversion_frames, axis=0, ignore_index=True)
conversion_df['conversions'] = 1
del conversion_frames  # make sure you delete these so they aren't sitting in memory

Finally, let's join the two datasets together.

In [ ]:
df = win_df.merge(conversion_df, on=['auction_id'], how='left').drop(['auction_id'], axis=1)
df = df.sort_values(by=['conversions'], ascending=False)
df.head()

## Data Cleansing
We now have a raw dataset, but we will want to apply some cleansing and filtering techniques before actually training the model.  This should make the model training steps later on much more accurate/efficient.

First, since we are dealing with mobile app data, we can drop all the columns that don't pertain to mobile:

In [ ]:
df = df.drop(['platform_browser', 'platform_browser_version', 'platform_device_type', 'domain', 'environment_type', 'platform_js', 'site_name', 'site_id'], axis=1)

We can also drop some columns which contain essentially duplicate data (we don't need app_bundle and app_name):

In [ ]:
df = df.drop(['app_name', 'app_id', 'geo_city', 'geo_zip', 'geo_country'], axis=1)

We also want to make a couple of column substitutions.  Bid Models doesn't support specific timestamps or lat/lon combinations but we can filter on "hour of day" and "lat/lon present":

In [ ]:
df['hour_of_day_utc'] = pd.to_datetime(df['bid_time_epoch_in_usecs'], unit='us')
df['hour_of_day_utc'] = df['hour_of_day_utc'].dt.hour
df['lat_long_present'] = pd.notna(df['geo_lat'])
df = df.drop(['bid_time_epoch_in_usecs', 'geo_lat', 'geo_lon'], axis = 1)

Next, we'll look for columns with a large number of missing values since they won't really be useful to our model.  For now, we'll drop any column where more than 30% of the values are missing.

In [ ]:
num_rows = len(df.index)
null_counts = df.isnull().sum()
cols_to_remove = null_counts[null_counts > (num_rows*.3)]
cols_to_remove = cols_to_remove.drop(['conversions'], axis=0)  # don't remove the conversion field, we need it!

df = df.drop(cols_to_remove.index, axis=1)
df.head()

Now let's look at data cardinality.  Basically, we'll eliminate the extreme outliers.  Anything that only has one value is useless to us and anything that has very very high cardinality will create clutter and probably hinder us from getting to a good model; we'll drop values from high cardinality fields that have very few impressions:

In [ ]:
df.apply(pd.Series.nunique)

We don't have any columns with a single value, so nothing to remove there.  We do have some high cardinality fields: `app_bundle`, `platform_device_model`, and `placement`.  Let's try to reduce that cardinality by replacing rarely recurring values with `null` (which will basically stand for "other").  The code below will replace any value that saw less than 10 impressions:

In [ ]:
for col in df.columns:
    if col in ['conversions']:
        continue
    value_counts = df[col].value_counts().reset_index()
    value_counts.columns = [col, col+'_freq']
    value_counts[col] = value_counts[col].astype(object)
    df = pd.merge(df, value_counts, on=col)
    
    mask = df[col+'_freq'] < 10
    df.loc[mask, col] = np.NaN
    df = df.drop(col+'_freq', axis=1)


df.apply(pd.Series.nunique)

Now we do have a few single value fields.  Let's drop these:

In [ ]:
df = df.drop(['auction_type', 'exchange_predicted_view_rate', 'video_boxing_allowed'], axis=1)
df.head()

Finally, let's aggregate the data base on all our keys, summing conversions:

In [ ]:
# add a column to count impressions
df['impressions'] = 1

# replace the empty conversion values with "0"
df['conversions'] = df['conversions'].replace(np.nan, 0)

# we want to group by all columns, with the exception of "conversions" and "impressions"
cols_to_groupby = list(df.columns)
cols_to_groupby.remove('conversions')
cols_to_groupby.remove('impressions')

# now we can aggregate
# note that pandas won't group by np.NaN so we replace with -1 and fix later
df = df.fillna(-1).groupby(cols_to_groupby).sum()

df = df.reset_index().sort_values(by=['conversions'], ascending=False)
df.head()

Okay, we have a clean set of training data to work with! In the next step, we'll start exploring the data a little more and try find the exact dataset that we want to train our model against.  First though, let's save our dataset so we don't have to re-load all the data again when we come back.

In [ ]:
df.to_pickle('data/step1.pkl')

A note on running this notebook: for a large number of input files, this step can take some time to run.  If you don't want to sit and wait, you can execute the notebook in the background using the `nbconvert` library.  To do this on a SageMaker Notebook Instance, open a terminal tab and execute the following commands:

```bash
$ echo ". /home/ec2-user/anaconda3/etc/profile.d/conda.sh" >> ~/.bashrc
$ source ~/.bashrc
$ conda activate python2
$ nohup jupyter nbconvert <path to notebook> --inplace --ExecutePreprocessor.kernel_name=python2 --ExecutePreprocessor.timeout=-1 --execute &
```

When this finishes, you can open the notebook in the GUI and see the results.